<a href="https://colab.research.google.com/github/Veronicaospina/Lab2Datos2/blob/main/Lab_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Instalaciones

In [ ]:
!pip install graphviz==0.20.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 617.9 kB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.3
    Uninstalling graphviz-0.20.3:
      Successfully uninstalled graphviz-0.20.3


In [ ]:
!pip install networkx==3.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 5.1 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.3
    Uninstalling networkx-3.3:
      Successfully uninstalled networkx-3.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1

In [ ]:
!pip install pygraphviz==1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.8/120.8 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for pygraphviz
  Running setup.py clean for pygraphviz
Failed to build pygraphviz
ERROR: Could not build wheels for pygraphviz, which is required to install pyproject.toml-based projects


In [ ]:
!pip install networkx==3.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 8.4 MB/s eta 0:00:00
  Attempting uninstall: networkx
    Found existing installation: networkx 3.1
    Uninstalling networkx-3.1:
      Successfully uninstalled networkx-3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.2.1+cu121 requires nvidia-cublas-cu12==12.1.3.1; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-cupti-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-nvrtc-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1+cu121 requires nvidia-cuda-runtime-cu12==12.1.105; platform_system == "Linux" and platform_machine == "x86_64", which is not installed.
torch 2.2.1

#Funciomaniento interno

In [ ]:
import pandas as pd
import numpy as np
import graphviz as gv
from typing import Any, List, Optional, Tuple
from pprint import pprint
from queue import Queue
import networkx as nx
import matplotlib.pyplot as plt
import heapq
import plotly.graph_objects as go
import plotly.express as px
import geopandas as gpd

In [ ]:
dataf = pd.read_csv("/content/flights_final.csv")
dataf.index = range(1, len(dataf) +1) #Para inciar el conteo del índice en 1 en lugar de 0
dataf.head()

,Source Airport Code,Source Airport Name,Source Airport City,Source Airport Country,Source Airport Latitude,Source Airport Longitude,Destination Airport Code,Destination Airport Name,Destination Airport City,Destination Airport Country,Destination Airport Latitude,Destination Airport Longitude
1,COK,Cochin International Airport,Kochi,India,10.152000,76.401901,SHJ,Sharjah International Airport,Sharjah,United Arab Emirates,25.328600,55.517200
2,GIG,Rio Galeão – Tom Jobim International Airport,Rio De Janeiro,Brazil,-22.809999,-43.250557,BSB,Presidente Juscelino Kubistschek International...,Brasilia,Brazil,-15.869167,-47.920834
3,TSF,Treviso-Sant'Angelo Airport,Treviso,Italy,45.648399,12.194400,BRI,Bari Karol Wojtyła Airport,Bari,Italy,41.138901,16.760599
4,CDG,Charles de Gaulle International Airport,Paris,France,49.012798,2.550000,FLR,Peretola Airport,Florence,Italy,43.810001,11.205100
5,CGH,Congonhas Airport,Sao Paulo,Brazil,-23.626110,-46.656387,CGB,Marechal Rondon Airport,Cuiaba,Brazil,-15.652900,-56.116699


In [ ]:
radio_tierra = 6371.0

# Convierte las coordenadas de grados a radianes
dataf['Latitud_salida_rad'] = np.radians(dataf['Source Airport Latitude'])
dataf['Longitud_salida_rad'] = np.radians(dataf['Source Airport Longitude'])
dataf['Latitud_llegada_rad'] = np.radians(dataf['Destination Airport Latitude'])
dataf['Longitud_llegada_rad'] = np.radians(dataf['Destination Airport Longitude'])

# Calcula las diferencias de latitud y longitud
dataf['dLat'] = dataf['Latitud_llegada_rad'] - dataf['Latitud_salida_rad']
dataf['dLon'] = dataf['Longitud_llegada_rad'] - dataf['Longitud_salida_rad']

# Calcula la distancia utilizando la fórmula de Haversine
dataf['distancia'] = 2 * np.arcsin(np.sqrt(
    np.sin(dataf['dLat'] / 2)**2 +
    np.cos(dataf['Latitud_salida_rad']) * np.cos(dataf['Latitud_llegada_rad']) *
    np.sin(dataf['dLon'] / 2)**2
)) * radio_tierra

In [ ]:
class Grafo:
    def __init__(self):
        self.graph = {}

    def add_edge(self, source, destination, distance):
        if source not in self.graph:
            self.graph[source] = {}
        if destination not in self.graph:
            self.graph[destination] = {}

        self.graph[source][destination] = distance
        self.graph[destination][source] = distance

    def __str__(self):
        result = ""
        for source in self.graph:
            for destination, distance in self.graph[source].items():
                result += f"({source} -- {destination} : {distance} km)\n"
        return result

    def dijkstra(self, airport1, airport2):
        if airport1 not in self.graph or airport2 not in self.graph:
            return None  # Al menos uno de los aeropuertos no existe en el Grafo

        distances = {vertex: float('inf') for vertex in self.graph}
        distances[airport1] = 0

        previous_vertices = {}  # Para mantener un seguimiento de los vértices anteriores en el camino mínimo
        priority_queue = [(0, airport1)]

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)

            if current_vertex == airport2:
                break

            if current_distance > distances[current_vertex]:
                continue

            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_vertices[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))

        if distances[airport2] == float('inf'):
            return None

        path = []
        current = airport2
        while current != airport1:
            path.append(current)
            current = previous_vertices[current]
        path.append(airport1)

        path.reverse()

        # Calcular y mostrar las distancias entre aeropuertos en el camino mínimo
        airport_distance = []
        for i in range(len(path) - 1):
            current_airport = path[i]
            next_airport = path[i + 1]
            distance_between_airports = self.graph[current_airport][next_airport]
            airport_distance.append(distance_between_airports)

        for i, airport in enumerate(path[:-1]):
            print(f"Aeropuerto: {airport} -> Aeropuerto: {path[i + 1]}, Distancia: {airport_distance[i]} km")

        return path

        #Función para mostrar los 10 más lejos
    def dijkstra_distance(self, airport1, airport2):
        if airport1 not in self.graph or airport2 not in self.graph:
            return None  # Al menos uno de los aeropuertos no existe en el Grafo

        distances = {vertex: float('inf') for vertex in self.graph}
        distances[airport1] = 0

        previous_vertices = {}  # Para mantener un seguimiento de los vértices anteriores en el camino mínimo
        priority_queue = [(0, airport1)]

        while priority_queue:
            current_distance, current_vertex = heapq.heappop(priority_queue)

            if current_vertex == airport2:
                break

            if current_distance > distances[current_vertex]:
                continue

            for neighbor, weight in self.graph[current_vertex].items():
                distance = current_distance + weight
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    previous_vertices[neighbor] = current_vertex
                    heapq.heappush(priority_queue, (distance, neighbor))

        if distances[airport2] == float('inf'):
            return None

        path = []
        current = airport2
        while current != airport1:
            path.append(current)
            current = previous_vertices[current]
        path.append(airport1)

        path.reverse()

        total_distance = 0
        for i in range(len(path) - 1):
            current_airport = path[i]
            next_airport = path[i + 1]
            distance_between_airports = self.graph[current_airport][next_airport]
            total_distance = total_distance+distance_between_airports;
        return total_distance

In [ ]:
flight_graph = Grafo()


# Iterar a través de las filas del DataFrame y agregar las aristas al Grafo
for index, row in dataf.iterrows():
    airport1 = row['Source Airport Code']
    airport2 = row['Destination Airport Code']
    distance = row['distancia']
    flight_graph.add_edge(airport1, airport2, distance)

# Imprimir el Grafo
print(flight_graph)

(COK -- SHJ : 2774.426364230329 km)
(COK -- DXB : 2782.5767918356187 km)
(COK -- KUL : 2912.8509214967808 km)
(COK -- MAA : 517.6618677279447 km)
(COK -- BOM : 1063.8121533213086 km)
(COK -- CMB : 505.6100167107124 km)
(COK -- MLE : 734.5624600321364 km)
(COK -- TRV : 194.19262804417656 km)
(COK -- DMM : 3330.837829067502 km)
(COK -- SIN : 3202.4664384046746 km)
(COK -- BLR : 367.26072156006865 km)
(COK -- HYD : 817.0601555722283 km)
(COK -- CCJ : 119.88820614927293 km)
(COK -- MCT : 2434.334973259793 km)
(COK -- DOH : 3108.301081405822 km)
(COK -- PNQ : 974.691902158979 km)
(COK -- DEL : 2048.904297611124 km)
(COK -- RUH : 3540.286559129541 km)
(COK -- AGX : 468.03964430418887 km)
(COK -- BAH : 3246.9894396172376 km)
(COK -- AUH : 2795.1367312249213 km)
(COK -- KWI : 3636.5236864893336 km)
(SHJ -- COK : 2774.426364230329 km)
(SHJ -- CGP : 3697.818986046822 km)
(SHJ -- DME : 3638.849909313691 km)
(SHJ -- TRV : 2939.5332212467824 km)
(SHJ -- AHB : 1542.790369655455 km)
(SHJ -- AUH : 132

In [ ]:
def getFlights (search:dict, dataf:pd.DataFrame) -> pd.DataFrame:

  flights = pd.DataFrame({col: [] for col in dataf.keys()})

  #Si no hay requisitos se devuelve el mismo dataframe
  if (len(search) == 0):
    return dataf

  for i in dataf.index:  #Recorre todo el dataframe y filtra aquellos que cumplan con el requisito
    if (all([any([dataf[col][i] == crit for crit in search[col]]) for col in search.keys()])):
      flight = pd.DataFrame({col: dataf[col][i] for col in dataf.keys()}, index=[len(flights)+1])
      flights = pd.concat([flights, flight])

  return flights

In [ ]:
#Sirve para obtener los nodos (aeropuertos) de un dataframe de vuelos
def searchAir (dataf:pd.DataFrame) -> pd.DataFrame:
  cols = [col for col in dataf.keys()]
  nodes_names_set = set()
  nodes_set = []

  #Busca los nodos no repetidos tanto de origen como destino
  for i in dataf.index:
    if (dataf['Source Airport Name'][i] not in nodes_names_set):
      nodes_names_set.add(dataf['Source Airport Name'][i])
      nodes_set.append([dataf[col][i] for col in cols[:6]])

    if (dataf['Destination Airport Name'][i] not in nodes_names_set):
      nodes_names_set.add(dataf['Destination Airport Name'][i])
      nodes_set.append([dataf[col][i] for col in cols[6:12]])


  cols = ['Airport Code', 'Airport Name',
          'Airport City', 'Airport Country',
          'Airport Latitude', 'Airport Longitude']

  #Converte los nodos a un nuevo dataframe
  nodes = pd.DataFrame({col: [] for col in cols})
  for node_set in nodes_set:
    node = pd.DataFrame({cols[i]: node_set[i] for i in range(len(node_set))}, index=[len(nodes)+1])
    nodes = pd.concat([nodes, node])

  return nodes

In [ ]:
def graphFlights (search:dict, dataf:pd.DataFrame, drawlines: bool) -> pd.DataFrame:
  #Token de cuenta de mapbox para ejecutar el mapa
  mapbox_access_token = 'pk.eyJ1Ijoic2ViYXN0aWFubWFsZG9uYWRvMTk0NSIsImEiOiJjbGluYnRobHkwbDQyM2xwOGc4aGN5ZnpvIn0.Jal1X7da0VhVK8gkKrWBng'

  flights = getFlights(search, dataf) #Se buscan los vuelos solicitados
  nodes = searchAir(flights)  #Se consiguen los aeropuertos

  lon = [nodes['Airport Longitude'][i] for i in nodes.index]
  lat = [nodes['Airport Latitude'][i] for i in nodes.index]

  fig = go.Figure()

  #Si se solicita dibujar lineas : [True] | [False]
  if (drawlines):
    for i in flights.index:
      fig.add_trace(go.Scattermapbox(
      mode = "lines",
      lon = [flights['Source Airport Longitude'][i], flights['Destination Airport Longitude'][i]],
      lat = [flights['Source Airport Latitude'][i], flights['Destination Airport Latitude'][i]],
      hoverinfo='skip',
      marker = {'size': 1}))

  #Dibujo de los nodos
  fig.add_trace(go.Scattermapbox(
      mode="markers+text",
      lon=lon, lat=lat,
      marker={'size': 20, 'symbol': ["airport" for _ in range(len(nodes))]},
      text=[f"{nodes['Airport Name'][i]} ({nodes['Airport Code'][i]}) ({nodes['Airport City'][i]}) ({nodes['Airport Country'][i]})" for i in nodes.index],
      textposition="bottom right"))

  fig.update_layout(
      mapbox = {
          'accesstoken': mapbox_access_token,
          'style': "outdoors", 'zoom': 0.7},
      showlegend = False)

  fig.show(renderer="colab")

In [ ]:
def graphAirports(dataf: pd.DataFrame) -> None:
    # Token de cuenta de Mapbox para ejecutar el mapa
    mapbox_access_token = 'pk.eyJ1Ijoic2ViYXN0aWFubWFsZG9uYWRvMTk0NSIsImEiOiJjbGluYnRobHkwbDQyM2xwOGc4aGN5ZnpvIn0.Jal1X7da0VhVK8gkKrWBng'

    nodes = searchAir(dataf)  # Se obtienen todos los aeropuertos

    lon = [nodes['Airport Longitude'][i] for i in nodes.index]
    lat = [nodes['Airport Latitude'][i] for i in nodes.index]

    fig = go.Figure()


    # Dibujo de los nodos
    fig.add_trace(go.Scattermapbox(
      mode="markers+text",
      lon=lon, lat=lat,
      marker={'size': 20, 'symbol': ["airport" for _ in range(len(nodes))]},
      text=[f"{nodes['Airport Name'][i]} ({nodes['Airport Code'][i]}) ({nodes['Airport City'][i]}) ({nodes['Airport Country'][i]})" for i in nodes.index],
      textposition="bottom right"))

    fig.update_layout(
        mapbox={
            'accesstoken': mapbox_access_token,
            'style': "outdoors", 'zoom': 0.7},
        showlegend=False)

    fig.show(renderer="colab")




In [ ]:
def graphAirportsPath(camino, dataf: pd.DataFrame) -> None:
    # Token de cuenta de Mapbox para ejecutar el mapa
    mapbox_access_token = 'pk.eyJ1Ijoic2ViYXN0aWFubWFsZG9uYWRvMTk0NSIsImEiOiJjbGluYnRobHkwbDQyM2xwOGc4aGN5ZnpvIn0.Jal1X7da0VhVK8gkKrWBng'
    flights = getFlights({}, dataf)
    nodes = searchAir(dataf)  # Se obtienen todos los aeropuertos
    lat=[]
    lon=[]
    for i in range(1, len(nodes['Airport Longitude'])):
        if nodes['Airport Code'][i] in camino:
            lon.append(nodes['Airport Longitude'][i])
            lat.append(nodes['Airport Latitude'][i])

    fig = go.Figure()
    j=1
    for g in range(len(camino)):
        sw=True
        k=1
        while k < len(flights.index) and sw==True and j < len(camino):
            if flights['Source Airport Code'][k] == camino[j-1] and flights['Destination Airport Code'][k] == camino[j]:
                j=j+1
                fig.add_trace(go.Scattermapbox(
                mode = "lines",
                lon = [flights['Source Airport Longitude'][k], flights['Destination Airport Longitude'][k]],
                lat = [flights['Source Airport Latitude'][k], flights['Destination Airport Latitude'][k]],
                hoverinfo='skip',
                marker = {'size': 1}))
                sw=False
            k=k+1

    # Dibujo de los nodos
    fig.add_trace(go.Scattermapbox(
        mode="markers+text",
        lon=lon, lat=lat,
        marker={'size': 20, 'symbol': ["airport" for _ in range(len(nodes))]},
        text=[f"{nodes['Airport Name'][i]} ({nodes['Airport Code'][i]}) ({nodes['Airport City'][i]}) ({nodes['Airport Country'][i]})" for i in nodes.index if nodes['Airport Code'][i] in camino],
        textposition="bottom right"))

    fig.update_layout(
        mapbox={
            'accesstoken': mapbox_access_token,
            'style': "outdoors", 'zoom': 0.7},
        showlegend=False)

    fig.show(renderer="colab")

#Generar mapa

In [ ]:
# Llama a la función para graficar todos los aeropuertos
graphAirports(dataf)

In [ ]:
graphFlights({'Source Airport Code': ['JNB']}, dataf, True)

#Aeropuertos más lejanos

In [ ]:
flag=True
while flag:
  # Solicitamos el codigo del aereopuerto a consultar
  source = input("Ingrese el código del aeropuerto: ").strip().upper()
  node_exists = source in dataf['Source Airport Code'].values
  if node_exists==False:
    print("Por favor ingrese un código de aeropuerto válido")
  else:
    # Obtemos un dataframe de los nodos
    flights = getFlights({}, dataf)
    nodes_dataf = searchAir(flights)
    # Creamos un dataframe auxiliar sin el aereopuerto que se esta consultando
    src_distances_dataf = nodes_dataf.copy()
    src_distances_dataf.drop(src_distances_dataf[src_distances_dataf['Airport Code'] == source].index,inplace=True)
    # Creamos una nueva columna para adjuntar las distancias de los caminos minimos
    src_distances_dataf['Distancia camino minimo'] = None
    src_distances_dataf.reset_index(inplace=True,drop=True)

    # Llenar la nueva columna con los valores respectivos
    for i in range(len(src_distances_dataf)):
        # Calculamos distancia total del camino minimo a partir de dijkstra
        distancia_total = flight_graph.dijkstra_distance(source, src_distances_dataf.at[i,'Airport Code'])
        if(distancia_total):
          src_distances_dataf.at[i, 'Distancia camino minimo'] = distancia_total

    # Ordenamos el dataframe de mayor a menor y obtemos las primeras filas
    src_distances_dataf = src_distances_dataf.sort_values(by='Distancia camino minimo', ascending=False)
    src_distances_dataf = src_distances_dataf.head(10)
    src_distances_dataf.reset_index(inplace=True,drop=True)

    # Imprimimos la informacion del dataframe restante
    for i in range(len(src_distances_dataf)):
      if i==0:
        print(f"Aeropuerto #{(i+1)} (Más lejano)")
      else:
        print(f"Aeropuerto #{(i+1)}")
      print(f"  - Codigo: {src_distances_dataf.at[i,'Airport Code']}")
      print(f"  - Nombre: {src_distances_dataf.at[i,'Airport Name']}")
      print(f"  - Ciudad: {src_distances_dataf.at[i,'Airport City']}")
      print(f"  - Pais: {src_distances_dataf.at[i,'Airport Country']}")
      print(f"  - Latitud: {src_distances_dataf.at[i,'Airport Latitude']}")
      print(f"  - Longitud: {src_distances_dataf.at[i,'Airport Longitude']}")
      numero_redondeado = round(src_distances_dataf.at[i, 'Distancia camino minimo'], ndigits=2)
      print(f"  - Distancia camino minimo: {numero_redondeado}")
      print(" ")
      flag=False

Ingrese el código del aeropuerto: MIA
Aeropuerto #1 (Más lejano)
  - Codigo: CCK
  - Nombre: Cocos (Keeling) Islands Airport
  - Ciudad: Cocos Keeling Island
  - Pais: Cocos (Keeling) Islands
  - Latitud: -12.18830013
  - Longitud: 96.83390045
  - Distancia camino minimo: 22371.76
 
Aeropuerto #2
  - Codigo: XCH
  - Nombre: Christmas Island Airport
  - Ciudad: Christmas Island
  - Pais: Christmas Island
  - Latitud: -10.45059967
  - Longitud: 105.6900024
  - Distancia camino minimo: 21387.1
 
Aeropuerto #3
  - Codigo: DCN
  - Nombre: RAAF Base Curtin
  - Ciudad: Derby
  - Pais: Australia
  - Latitud: -17.58139992
  - Longitud: 123.8280029
  - Distancia camino minimo: 20556.06
 
Aeropuerto #4
  - Codigo: MKQ
  - Nombre: Mopah Airport
  - Ciudad: Merauke
  - Pais: Indonesia
  - Latitud: -8.520290375
  - Longitud: 140.4179993
  - Distancia camino minimo: 20455.12
 
Aeropuerto #5
  - Codigo: OKL
  - Nombre: Oksibil Airport
  - Ciudad: Oksibil
  - Pais: Indonesia
  - Latitud: -4.9071
  - Lo

#Dados dos puntos camino mínimo


In [ ]:
flight_graph = Grafo()
for _, row in dataf.iterrows():
    airport1 = row['Source Airport Code'].strip().upper()
    airport2 = row['Destination Airport Code'].strip().upper()
    distance = row['distancia']
    flight_graph.add_edge(airport1, airport2, distance)
flag1=True
while flag1:
  airport1 = input("Ingrese el código del aeropuerto de origen: ").strip().upper()
  node_exists1 = airport1 in dataf['Source Airport Code'].values
  if node_exists1==False:
    print("Por favor ingrese un código de aeropuerto válido")
  else:
    flag2=True
    while flag2:
      airport2 = input("Ingrese el código del aeropuerto de destino: ").strip().upper()
      node_exists2 = airport2 in dataf['Source Airport Code'].values
      if node_exists2==False:
        print("Por favor ingrese un código de aeropuerto válido")
      else:
        camino_minimo = flight_graph.dijkstra(airport1, airport2)
        if camino_minimo:
            print("Camino mínimo:")
            print(" -> ".join(camino_minimo))
            graphAirportsPath(camino_minimo,dataf)

        else:
            print("No se encontró un camino entre los aeropuertos especificados.")
        flag2=False
    flag1=False

Ingrese el código del aeropuerto de origen: YES
Ingrese el código del aeropuerto de destino: NOP
Aeropuerto: YES -> Aeropuerto: THR, Distancia: 555.1364310345972 km
Aeropuerto: THR -> Aeropuerto: TBZ, Distancia: 526.8998011115053 km
Aeropuerto: TBZ -> Aeropuerto: IST, Distancia: 1533.0043879780965 km
Aeropuerto: IST -> Aeropuerto: NOP, Distancia: 530.9786644328566 km
Camino mínimo:
YES -> THR -> TBZ -> IST -> NOP
